Importando Pacotes

In [32]:
# !pip install -r requirements.txt

In [33]:
import pandas as pd
import requests
import json
import time
from pathlib import Path

Configurações e variaveis gerais do código  

Documentação API: https://www.api-football.com/documentation-v3

In [34]:
#Exibir totas as colunas no pandas
pd.set_option('display.max_columns', None)

base_url = "https://v3.football.api-sports.io"
headers = {
  'x-rapidapi-key': '75a1d666035ab70c3321c2884333be2b',
  'x-rapidapi-host': 'v3.football.api-sports.io'
}

0 - Variaveis da Coleta

In [35]:
season = 2022
id_liga = 71 #Brasileirao Serie A
limite_request_dia = 100 #Limitando o numero de chamadas devido caracteristica da API Free
limite_request_minuto = 10

path_output = "data/raw/brasileirao/"

1 - Get partidas do campeonato

In [36]:
url_partidas = f"{base_url}/fixtures?timezone=America/Sao_Paulo&league={id_liga}&season={season}"
response_partidas = requests.get(url_partidas, headers=headers).json()

In [37]:
df_partidas = pd.json_normalize(response_partidas['response'])
df_partidas['fixture.date'] = pd.to_datetime(df_partidas['fixture.date'])

In [38]:
df_partidas.head()

,fixture.id,fixture.referee,fixture.timezone,fixture.date,fixture.timestamp,fixture.periods.first,fixture.periods.second,fixture.venue.id,fixture.venue.name,fixture.venue.city,fixture.status.long,fixture.status.short,fixture.status.elapsed,league.id,league.name,league.country,league.logo,league.flag,league.season,league.round,teams.home.id,teams.home.name,teams.home.logo,teams.home.winner,teams.away.id,teams.away.name,teams.away.logo,teams.away.winner,goals.home,goals.away,score.halftime.home,score.halftime.away,score.fulltime.home,score.fulltime.away,score.extratime.home,score.extratime.away,score.penalty.home,score.penalty.away
0,837991,Bruno Arleu de Araujo,America/Sao_Paulo,2022-04-10 16:00:00-03:00,1649617200,1649617200,1649620800,234.0,Estádio Governador Magalhães Pinto,"Belo Horizonte, Minas Gerais",Match Finished,FT,90,71,Serie A,Brazil,https://media-4.api-sports.io/football/leagues...,https://media-4.api-sports.io/flags/br.svg,2022,Regular Season - 1,1062,Atletico-MG,https://media-4.api-sports.io/football/teams/1...,True,119,Internacional,https://media-4.api-sports.io/football/teams/1...,False,2,0,1,0,2,0,None,None,None,None
1,837992,Anderson Daronco,America/Sao_Paulo,2022-04-09 16:30:00-03:00,1649532600,1649532600,1649536200,NaN,Estadio Jornalista Mário Filho,"Rio de Janeiro, Rio de Janeiro",Match Finished,FT,90,71,Serie A,Brazil,https://media-4.api-sports.io/football/leagues...,https://media-4.api-sports.io/flags/br.svg,2022,Regular Season - 1,124,Fluminense,https://media-4.api-sports.io/football/teams/1...,None,128,Santos,https://media-4.api-sports.io/football/teams/1...,None,0,0,0,0,0,0,None,None,None,None
2,837993,Wagner do Nascimento Magalhaes,America/Sao_Paulo,2022-04-10 19:00:00-03:00,1649628000,1649628000,1649631600,NaN,Estádio Cícero Pompeu de Toledo,"São Paulo, São Paulo",Match Finished,FT,90,71,Serie A,Brazil,https://media-4.api-sports.io/football/leagues...,https://media-4.api-sports.io/flags/br.svg,2022,Regular Season - 1,126,Sao Paulo,https://media-4.api-sports.io/football/teams/1...,True,134,Atletico Paranaense,https://media-4.api-sports.io/football/teams/1...,False,4,0,1,0,4,0,None,None,None,None
3,837994,Caio Max Augusto Vieira,America/Sao_Paulo,2022-04-09 21:00:00-03:00,1649548800,1649548800,1649552400,258.0,Allianz Parque,"São Paulo, São Paulo",Match Finished,FT,90,71,Serie A,Brazil,https://media-4.api-sports.io/football/leagues...,https://media-4.api-sports.io/flags/br.svg,2022,Regular Season - 1,121,Palmeiras,https://media-4.api-sports.io/football/teams/1...,False,129,Ceara,https://media-4.api-sports.io/football/teams/1...,True,2,3,1,2,2,3,None,None,None,None
4,837995,Wilton Pereira Sampaio,America/Sao_Paulo,2022-04-10 16:00:00-03:00,1649617200,1649617200,1649620800,218.0,Estádio Nilton Santos,Rio de Janeiro,Match Finished,FT,90,71,Serie A,Brazil,https://media-4.api-sports.io/football/leagues...,https://media-4.api-sports.io/flags/br.svg,2022,Regular Season - 1,120,Botafogo,https://media-4.api-sports.io/football/teams/1...,False,131,Corinthians,https://media-4.api-sports.io/football/teams/1...,True,1,3,0,3,1,3,None,None,None,None


Output Arquivo

In [39]:
df_partidas.to_parquet(path_output + f'partidas/partidas_{season}.parquet')

2 - Get Estatisticas das partidas do campeonato

In [40]:
#Todas as partidas com status Finalizado (FT)
todas_partidas = set(df_partidas[df_partidas['fixture.status.short'] == 'FT']['fixture.id'])

#Partidas já coletadas
try:
    partidas_coletadas = set(pd.read_parquet(f'{path_output}/partidas_estatistica/{season}')['fixture.id'])
except:
    partidas_coletadas = set()

#Gera lista de partidas novas à serem coletadas, limitando ao máximo permitido pela API
id_partidas_coleta = list(todas_partidas - partidas_coletadas)[:limite_request_dia]


print('todas_partidas', len(todas_partidas), '\n',
      'partidas_coletadas', len(partidas_coletadas), '\n',
      'qtd partidas a coletar: ', len(id_partidas_coleta)
      )

todas_partidas 380 
 partidas_coletadas 0 
 qtd partidas a coletar:  100


Consulta e Output arquivos

In [41]:
print('limite_minuto', limite_request_minuto, ' | ','limite_dia', limite_request_dia)

for i, id in enumerate(id_partidas_coleta):
    try:
        #Aguarda 1 min se limite requisicao atingido
        if (i+1) % limite_request_minuto == 0:
            print('modo de espera...')
            time.sleep(60)

        #coleta API
        url_statistics = f"{base_url}/fixtures/statistics?fixture={id}"
        response_statistics = requests.get(url_statistics, headers=headers).json()

        #Verifica se há erro na requisicao
        erros = response_statistics['errors']
        if len(erros) > 0:
            raise Exception(erros)
            
        #Criação do Data Frame
        df_estatistica = pd.json_normalize(response_statistics['response'],
                                        record_path="statistics",
                                        meta=[["team", "id"], ["team", "name"]]
                                        ).astype('str')
        df_estatistica['fixture.id'] = id

        #Output DataFrame
        output_dir = Path(f'{path_output}/partidas_estatistica/{season}/')
        output_file = f'{output_dir}/partida_{id}.parquet'
        output_dir.mkdir(parents=True, exist_ok=True)

        df_estatistica.to_parquet(output_file)
        print(i+1, output_file)
    
    except Exception as e:
        output_json_file = f'{path_output}/partidas_estatistica_erros/partida_{id}.json'
        with open(output_json_file, 'w') as f:
            json.dump(response_statistics, f)
        pass

limite_minuto 10  |  limite_dia 100
obtido resultado da partida 837991
1 data\raw\brasileirao\partidas_estatistica\2022/partida_837991.parquet
obtido resultado da partida 837992
2 data\raw\brasileirao\partidas_estatistica\2022/partida_837992.parquet
obtido resultado da partida 837993
3 data\raw\brasileirao\partidas_estatistica\2022/partida_837993.parquet
obtido resultado da partida 837994
4 data\raw\brasileirao\partidas_estatistica\2022/partida_837994.parquet
obtido resultado da partida 837995
5 data\raw\brasileirao\partidas_estatistica\2022/partida_837995.parquet
obtido resultado da partida 837996
6 data\raw\brasileirao\partidas_estatistica\2022/partida_837996.parquet
obtido resultado da partida 837997
7 data\raw\brasileirao\partidas_estatistica\2022/partida_837997.parquet
obtido resultado da partida 837998
8 data\raw\brasileirao\partidas_estatistica\2022/partida_837998.parquet
obtido resultado da partida 837999
9 data\raw\brasileirao\partidas_estatistica\2022/partida_837999.parquet
m

KeyboardInterrupt: 

Odds Pre Match

In [42]:
url_odd_pre = f"{base_url}/odds/?fixture=1005649"
response_odds_pre = requests.get(url_odd_pre, headers=headers).json()

In [ ]:
df_odds = pd.json_normalize(response_odds_pre['response'])

Teste Leitura Arquivos

In [ ]:
df_teste_partidas = pd.read_parquet(path_output + f'partidas/partidas_{season}.parquet')
df_teste_partidas.head()

,fixture.id,fixture.referee,fixture.timezone,fixture.date,fixture.timestamp,fixture.periods.first,fixture.periods.second,fixture.venue.id,fixture.venue.name,fixture.venue.city,fixture.status.long,fixture.status.short,fixture.status.elapsed,league.id,league.name,league.country,league.logo,league.flag,league.season,league.round,teams.home.id,teams.home.name,teams.home.logo,teams.home.winner,teams.away.id,teams.away.name,teams.away.logo,teams.away.winner,goals.home,goals.away,score.halftime.home,score.halftime.away,score.fulltime.home,score.fulltime.away,score.extratime.home,score.extratime.away,score.penalty.home,score.penalty.away
0,1005649,Rodrigo José Pereira de Lima,America/Sao_Paulo,2023-04-16 16:00:00-03:00,1681671600,1681671600,1681675200,NaN,Estadio Jornalista Mário Filho,"Rio de Janeiro, Rio de Janeiro",Match Finished,FT,90,71,Serie A,Brazil,https://media-4.api-sports.io/football/leagues...,https://media-4.api-sports.io/flags/br.svg,2023,Regular Season - 1,127,Flamengo,https://media-4.api-sports.io/football/teams/1...,True,147,Coritiba,https://media-4.api-sports.io/football/teams/1...,False,3,0,1,0,3,0,None,None,None,None
1,1005650,Ramon Abatti,America/Sao_Paulo,2023-04-15 18:30:00-03:00,1681594200,1681594200,1681597800,218.0,Estádio Nilton Santos,Rio de Janeiro,Match Finished,FT,90,71,Serie A,Brazil,https://media-4.api-sports.io/football/leagues...,https://media-4.api-sports.io/flags/br.svg,2023,Regular Season - 1,120,Botafogo,https://media-4.api-sports.io/football/teams/1...,True,126,Sao Paulo,https://media-4.api-sports.io/football/teams/1...,False,2,1,1,1,2,1,None,None,None,None
2,1005651,Paulo Cesar Zanovelli da Silva,America/Sao_Paulo,2023-04-15 16:00:00-03:00,1681585200,1681585200,1681588800,258.0,Allianz Parque,"São Paulo, São Paulo",Match Finished,FT,90,71,Serie A,Brazil,https://media-4.api-sports.io/football/leagues...,https://media-4.api-sports.io/flags/br.svg,2023,Regular Season - 1,121,Palmeiras,https://media-4.api-sports.io/football/teams/1...,True,1193,Cuiaba,https://media-4.api-sports.io/football/teams/1...,False,2,1,1,1,2,1,None,None,None,None
3,1005652,Anderson Daronco,America/Sao_Paulo,2023-04-16 16:00:00-03:00,1681671600,1681671600,1681675200,11531.0,Neo Química Arena,"São Paulo, São Paulo",Match Finished,FT,90,71,Serie A,Brazil,https://media-4.api-sports.io/football/leagues...,https://media-4.api-sports.io/flags/br.svg,2023,Regular Season - 1,131,Corinthians,https://media-4.api-sports.io/football/teams/1...,True,135,Cruzeiro,https://media-4.api-sports.io/football/teams/1...,False,2,1,0,0,2,1,None,None,None,None
4,1005653,Maguielson Lima Barbosa,America/Sao_Paulo,2023-04-15 18:30:00-03:00,1681594200,1681594200,1681597800,220.0,Estádio Nabi Abi Chedid,"Bragança Paulista, São Paulo",Match Finished,FT,90,71,Serie A,Brazil,https://media-4.api-sports.io/football/leagues...,https://media-4.api-sports.io/flags/br.svg,2023,Regular Season - 1,794,RB Bragantino,https://media-4.api-sports.io/football/teams/7...,True,118,Bahia,https://media-4.api-sports.io/football/teams/1...,False,2,1,0,1,2,1,None,None,None,None


In [ ]:
df_teste_estatistica = pd.read_parquet(path_output + f'partidas_estatistica/{season}')
df_teste_estatistica.head(20)

,type,value,team.id,team.name,fixture.id
0,Shots on Goal,7,127,Flamengo,1005649
1,Shots off Goal,5,127,Flamengo,1005649
2,Total Shots,15,127,Flamengo,1005649
3,Blocked Shots,3,127,Flamengo,1005649
4,Shots insidebox,5,127,Flamengo,1005649
5,Shots outsidebox,10,127,Flamengo,1005649
6,Fouls,20,127,Flamengo,1005649
7,Corner Kicks,6,127,Flamengo,1005649
8,Offsides,1,127,Flamengo,1005649
9,Ball Possession,59%,127,Flamengo,1005649
